# Using NannyML Performance Estimation Algorithm from AWS Marketplace 

## Overview

NannyML can estimate the performance of a machine learning model running in production. You can read more about how it works [here](https://nannyml.readthedocs.io/en/stable/how_it_works/performance_estimation.html).


This sample notebook shows you how to use [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o) from AWS Marketplace to estimate the performance of your deployed machine learnign models.

Performance Estimation can work for binary classification, multiclass classification and regression.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o). 

## Contents
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure and visualize reference dataset](#B.-Configure-and-visualize-reference-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train the machine learning algorithm](#3:-Train-the-machine-learning-algorithm)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-the-algorithm)
1. [Deploy model](#4:-Deploy-model)
1. [Perform Batch Inference](#5.-Perform-Batch-Inference)
1. [Clean-up](#6.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o)
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

<font color='red'>Directly copy your assigned ARN code below:<font>

In [ ]:
algo_arn = "<Customer to specify algorithm ARN corresponding to their AWS region>"


## 2. Prepare dataset

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role
import pandas as pd
import json

### A. Dataset format expected by the algorithm

To fully demonstrate the capabilities of NannyML's performance estimation we will provide code for all 3 supported machine learning problem types.

- For Binary Classification we are going to use [NannyML's synthetic car loan dataset](https://nannyml.readthedocs.io/en/stable/datasets/binary_car_loan.html).
- For Multiclass Classification we are going to use [NannyML's synthetic multiclass creadit card assignment dataset](https://nannyml.readthedocs.io/en/stable/datasets/multiclass.html).
- For Regression we are going to use [NannyML's synthetic car price dataset](https://nannyml.readthedocs.io/en/stable/datasets/regression.html).


You can find some information about dataset format in **Usage Information** section of [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o).
<br>
More detailed information can be found in [NannyML's Data Requirements Documentation](https://nannyml.readthedocs.io/en/stable/tutorials/data_requirements.html).

<font color='red'>Edit code below as appropriate for the Machine Learning problem type you are interested in:<font>

In [ ]:
machine_learning_problem_type = "Binary Classification"
# machine_learning_problem_type = "Multiclass Classification"
# machine_learning_problem_type = "Regression"

### B. Configure and visualize reference dataset

In [ ]:
if machine_learning_problem_type == "Binary Classification":
    reference_dataset = "data/synthetic_car_loan_reference.csv"
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

# Show selected dataset
pd.read_csv(reference_dataset).head()

### C. Upload datasets to Amazon S3

In [ ]:
sagemaker_session = sage.Session()
bucket = sagemaker_session.default_bucket()
# Uncomment below to see selected default bucket
# bucket

In [ ]:
demo_prefix = "doc-notebook-demo"

In [ ]:
reference_data = sagemaker_session.upload_data(
    reference_dataset, bucket=bucket, key_prefix=demo_prefix
)

## 3: Train the machine learning algorithm

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [ ]:
role = get_execution_role()

In [ ]:
output_location = f"s3://{bucket}/{demo_prefix}/output"
# Uncomment below to see specified output location for Algorithm artifacts and outputs
# output_location

### 3.2 Train the algorithm

You can also find more information about dataset format in **Hyperparameters** section of [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o).

For even more detailed information you read NannyML tutorials on performance estimation for:
- [Binary Classification](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/binary_performance_estimation.html)
- [Multiclass Classification](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/multiclass_performance_estimation.html)
- [Regression](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/regression_performance_estimation.html)

In [ ]:
# Define hyperparameters
if machine_learning_problem_type == "Binary Classification":
    nannyml_parameters = {
        "y_pred_proba": "y_pred_proba",
        "y_pred": "y_pred",
        "y_true": "repaid",
        "timestamp_column_name": "timestamp",
        "metrics": ["roc_auc"],
        "chunk_size": 5000,
        "problem_type": "classification_binary",
        "model_inputs": [],
    }
    # json.dumps needed due to sagemaker specifications
    sagemaker_hyperparameters = {
        "data_filename": reference_dataset.split("/")[-1],
        "data_type": "csv",
        "problem_type": "classification_binary",
        "parameters": json.dumps(nannyml_parameters),
    }
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [ ]:
# Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name='nml-concept-shift',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    hyperparameters=sagemaker_hyperparameters,
)

In [ ]:
# Run the training job.
estimator.fit(
    {'training': reference_data}
)

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

## 4: Deploy model

**NannyML's Performance Estimation is not designed for real time inference, therefore it is not recommended to use it in this way.**

For this reason we are not showcasing the real-time inference feature of Sagemaker Algorithm.

## 5. Perform Batch Inference

In this section, you will perform batch inference using multiple input payloads together.

In [ ]:
# upload the batch-transform job input files to S3

if machine_learning_problem_type == "Binary Classification":
    inference_dataset = "data/synthetic_car_loan_analysis_with_targets.csv"
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

inference_data = sagemaker_session.upload_data(inference_dataset, bucket=bucket, key_prefix=demo_prefix)
# print("Transform input uploaded to " + inference_data)

In [ ]:
# Run the batch-transform job
transformer = estimator.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=output_location
)
transformer.transform(inference_data, content_type="text/csv")
transformer.wait()

**View Results of Performance Estimation**

In [ ]:
results = pd.read_csv(transformer.output_path + "/" + inference_dataset.split("/")[-1] + ".out", header = [0,1])
results

## 6. Clean-up

### A. Delete the model

In [ ]:
transformer.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

